# Colombia Análisis

In [12]:
#!pip install os numpy pandas
#!pip install openpyxl
import os
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

Se establece el working directory, los paths de otras carpetas y la fecha de análisis

In [13]:
os.chdir("/Users/JuanJose/Library/CloudStorage/OneDrive-UniversidaddelosAndes/Econometria/J&J/100652_J_J/Colombia/Stata")
Path_Input_xlsx = "DataFiles/Input/XLS/"
Path_working = "DataFiles/Working/"
Path_out_xlsx = "DataFiles/Output/XLS/"
#_date=datetime.datetime(2021, 12,1)
_Date=datetime(2021,12,1)+relativedelta(months=1)-timedelta(days=1)
#_Date=_Date.strftime("%d/%m/%Y")
_date=date.today()
_date=_date.strftime("%d/%m/%Y")

In [14]:
#funcion para ordenar variables
def reorder_columns(dataframe, col_names, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string list): columns names to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    Col_names=list()
    Col_names.extend(col_names)
    col_names=[]
    col_names.extend(Col_names)
    n=len(col_names)
    
    for i in range(n-1,-1,-1):
        col_name = col_names[i]
        temp_col = dataframe[col_name]
        dataframe = dataframe.drop(columns=[col_name])
        dataframe.insert(loc=position, column=col_name, value=temp_col)

    return dataframe

## Importar la base de datos

In [15]:
Colombia = pd.read_csv(Path_working+'Colombia_preliminar.csv',low_memory=False)
variablesColombia= list(Colombia.columns)
Colombia=Colombia.drop(columns='Unnamed: 0')
Colombia['id'].unique().size==len(Colombia) # cada obs tiene un id y es un procedimiento (paciente, FechaCirugia, Medico)

True

Se mantienen las variables importantes para el reshape

In [16]:
FechaControles=[control for control in variablesColombia if 'date_C' in control]
ids=['id','FechaCirugia']
varsToKeep=ids+FechaControles
Colombia_long=Colombia[varsToKeep]

Se hace un reshape long de la base de datos para las fechas de los controles

In [17]:
Colombia_long=Colombia_long.melt(id_vars=ids,value_vars=FechaControles,var_name='Control',value_name='FechaControl')
Colombia_long=Colombia_long[Colombia_long['FechaControl'].notna()]#se elimnan los valores Nan
Colombia_long=Colombia_long.sort_values(by=['id','Control','FechaControl'])#se ordena la base de datos

se convierten a fechas las variables de la fecha de la cirugia y la fecha del control, para luego crear la variable de los días transcurridos desde la cirugía hasta el control ('dias')

In [18]:
Colombia_long['FechaControl']=pd.to_datetime(Colombia_long['FechaControl'])
Colombia_long['FechaCirugia']=pd.to_datetime(Colombia_long['FechaCirugia'])
Colombia_long['dias']=Colombia_long['FechaControl']-Colombia_long['FechaCirugia']

Se genera la variable 'NuevoControl' en la que se verifica que el control si está entre los días predeterminados para ese control especifico. si la fecha no cae entre los días predeterminados para algún control, se pone como 0, para luego eliminarlos

In [19]:
conditions = [
    (Colombia_long['dias'] == timedelta(days=0)),#C0
    (Colombia_long['dias'] >= timedelta(days=20)) & (Colombia_long['dias'] <= timedelta(days=40)), #C1
    (Colombia_long['dias'] >= timedelta(days=70)) & (Colombia_long['dias'] <= timedelta(days=110)), #C2
    (Colombia_long['dias'] >= timedelta(days=150)) & (Colombia_long['dias'] <= timedelta(days=210)), #C3
    (Colombia_long['dias'] >= timedelta(days=305)) & (Colombia_long['dias'] <= timedelta(days=425)), #C4
    (Colombia_long['dias'] >= timedelta(days=540)) & (Colombia_long['dias'] <= timedelta(days=912)), #C5
    (Colombia_long['dias'] >= timedelta(days=913)) & (Colombia_long['dias'] <= timedelta(days=1277))] #C6
    #(Colombia_long['dias'] > timedelta(days=1277))] #C7, el control 7 son puros Nan
choices = ['date_C0','date_C1','date_C2','date_C3','date_C4','date_C5','date_C6']#hace falta el 7, pero el control 7 son puros Nan
Colombia_long['NuevoControl'] = np.select(conditions, choices,default=np.nan)
Colombia_long=Colombia_long[Colombia_long['NuevoControl'].notna()==True]

Se crea una base de datos auxiliar para saber cuales son las observaciones que cambiaron de control 

In [20]:
aux= Colombia_long.copy()
aux['diferentes']=aux['NuevoControl']!=aux['Control']
aux['diferentes'].sum()
aux['DuplicadosControl']=aux.duplicated(subset=['id','NuevoControl'])

Se crea la variable PromedioDiasControl, que son la cantidad de días a los cuales normalmente debería darse cada control. y luego, se crea la variable 'diff' que es la diferencia entre los días transcurridos entre la cirugía y el control, y los días que deberían haber transcurrido

In [21]:
conditions=[Colombia_long['NuevoControl']=='date_C0',Colombia_long['NuevoControl']=='date_C1',
Colombia_long['NuevoControl'] == 'date_C2',Colombia_long['NuevoControl'] == 'date_C3',
Colombia_long['NuevoControl'] == 'date_C4',Colombia_long['NuevoControl'] == 'date_C5',
Colombia_long['NuevoControl'] == 'date_C6'] #,Colombia_long['NuevoControl'] == 'date_C7'] el control 7 son puros Nan
choices=[timedelta(days=0), timedelta(days=30), timedelta(days=90), timedelta(days=180), timedelta(days=365), timedelta(days=730), timedelta(days=1095)]
Colombia_long['PromedioDiasControl'] = np.select(conditions, choices)
Colombia_long['PromedioDiasControl']=Colombia_long['PromedioDiasControl'].astype('timedelta64[ns]')
Colombia_long['diff']=abs(Colombia_long['PromedioDiasControl']-Colombia_long['dias'])

Dado que hay varias observaciones con el mismo control pues pertenencen al mismo rango de fechas posibles del control, se deja la que está más cercana los días que deberían haber transcurrido

In [22]:
Colombia_long=Colombia_long.loc[Colombia_long.groupby(['id','NuevoControl'])['diff'].idxmin()].reset_index(drop=True)

Se hace un merge con la infromación general de toda la base de datos de colombia, tanto las otras variables de controles como las variables de información del paciente y la cirugía 

In [23]:
Colombia_long=Colombia_long.merge(right=Colombia,on='id',validate='m:1')

Se crea una lista de las variables de los controles, y se añaden esas columnas a la base de datos, para generar una info condensada de cada control (del tipo long)

In [24]:
varsControl=[
'date','NoAsistio','Peso','IMC','IMCR','PPP','Ppeso','EWL', # no asistio y variables de peso
'DT2','Hipert','Depre','AdS','Dislip','DoM','ERGE','MalN','comor','RdTV_Alto', 'RdTV_Medio', 'RdTV_Bajo', #comorbilidades
'Fuga','ScT','ScR','Obstr','PaF','TrV','TrP','TnpUCI','ReH','NhC','Fist','Reint', #complicaciones
'mDT2','moDT2','moDT2num','iDT2','mHipert','mDepre','mDislip','mDoM','mERGE','mHipo','mHiper','AdS_poli','AdS_CPAP', #medicamentos
'PaF_fecha','TnpUCI_ingreso','TnpUCI_salida','ReH_ingreso','ReH_salida'#otras fechas
]
Colombia_long = Colombia_long.reindex(columns=Colombia_long.columns.tolist() + varsControl)

Para cada control se llena la información en las nuevas variables de control, y se eliminan las variables viejas

In [25]:
for i in range(8):
    for v in varsControl:
        for var in Colombia_long.columns.to_list():
            if v+'_C'+str(i)==var:
                Colombia_long.loc[(Colombia_long.Control=='date_C'+str(i)), v] = Colombia_long[var]
                Colombia_long=Colombia_long.drop(columns=var)

In [26]:
varsControl=['Peso','IMC','IMCR','PPP','Ppeso','EWL', # no asistio y variables de peso
'DT2','Hipert','Depre','AdS','Dislip','DoM','ERGE','MalN','RdTV_Alto', 'RdTV_Medio', 'RdTV_Bajo', #comorbilidades
'Fuga','ScT','ScR','Obstr','PaF','TrV','TrP','TnpUCI','ReH','NhC','Fist','Reint', #complicaciones
'mDT2','moDT2','moDT2num','iDT2','mHipert','mDepre','mDislip','mDoM','mERGE','mHipo','mHiper','AdS_poli','AdS_CPAP', #medicamentos
'PaF_fecha','TnpUCI_ingreso','TnpUCI_salida','ReH_ingreso','ReH_salida'#otras fechas
]
Colombia_long['SinInfo']=Colombia_long[varsControl].isna().sum(axis=1)==len(varsControl)

In [27]:
Colombia_long=Colombia_long[Colombia_long['SinInfo']==False]
Colombia_long=Colombia_long.drop(columns='SinInfo')

In [28]:
Colombia_long=Colombia_long.drop(columns='Control')